# join_2dataframes

In [2]:
%load_ext autoreload
%autoreload 2

# The autoreload extension is already loaded. To reload it, use:
#  %reload_ext autoreload

In [3]:
import os
import sys
import numpy as np
import pandas as pd

pd.options.display.float_format = '{:8,.2f}'.format

In [4]:
# mylib:
my_library = os.path.expanduser('~/.myconfigs')
my_spark = os.path.expanduser('~/spark2_dfanalysis')
sys.path.append(my_library)
sys.path.append(my_spark)

In [5]:
import pyspark as spark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

# spark = SparkSession.builder.appName('myappname').getOrCreate()
# print(spark)

In [6]:
# print(dir(pyspark))
# print(dir(pyspark.sql))
# print(dir(pyspark.rdd))

In [7]:
from shared.app_context import *

ctx = ApplicationContext("Dev-Job")
print(ctx.spark)

In [8]:
# print(sys.path)
from builder.DataFrameBuild import *

## Samples:

In [21]:
path_in  = "C:/Users/MerzDal/Documents/neighborhood/nas_epsilon/data/validation/sample_pqt"
path_out = "C:/Users/MerzDal/Documents/neighborhood/nas_epsilon/data/validation/sample_csv"

In [22]:
df = ctx.spark.read.parquet(path_in)

In [23]:
df.limit(4).toPandas()

,index,name,month,year,age,net_worth
0,0,steve,March,1920,99,"63,801.08"
1,1,steve,November,1986,33,"51,308.30"
2,2,sarah,October,2001,18,"24,108.56"
3,3,mary,April,1924,95,"79,342.94"


In [24]:
print(path_out)

C:/Users/MerzDal/Documents/neighborhood/nas_epsilon/data/validation/sample_csv


In [25]:
df.coalesce(1)\
.write\
.option("sep", ",")\
.option("header", "true")\
.mode("overwrite")\
.csv(path_out)

## DataFrameBuild Class:
### x = DataFrameBuild(ctx.spark)

In [8]:
x = DataFrameBuild(ctx.spark)

DF1

In [9]:
x = DataFrameBuild(ctx.spark)
num = 500
df1 = x.arrays_to_dataframe([[int(x) for x in np.linspace(1,num,num)],
                             x.build_array("string",num=num,width=8),
                             x.build_array("integer",num=num,nrange=(1,4)),
                             x.build_array("integer",num=num,nrange=(1,12)),
                             x.build_array("double",num=num,nrange=(0.0,10000))],
                            ['index','passwords','quarter','month','price'])

DF2

In [10]:
x = DataFrameBuild(ctx.spark)
num = 500
df2 = x.arrays_to_dataframe([[int(x) for x in np.linspace(1,num,num)],
                             x.build_array("string",num=num,width=8),
                             x.build_array("integer",num=num,nrange=(1,4)),
                             x.build_array("integer",num=num,nrange=(1,12)),
                             x.build_array("double",num=num,nrange=(0.0,10000))],
                             ['index','passwords','quarter','month','price'])

In [11]:
df1.limit(10).toPandas()

,index,passwords,quarter,month,price
0,1,aqamlptz,2,12,"2,185.37"
1,2,oxxxfham,4,7,"8,052.39"
2,3,btgdupsv,1,5,"9,916.70"
3,4,ggatzsbr,4,1,"2,274.50"
4,5,mcebalrm,4,11,157.24
5,6,wvvfbegj,2,5,"4,019.75"
6,7,yebxqdnd,4,8,"7,342.47"
7,8,evtrhyip,1,3,"5,902.27"
8,9,mtdqogmt,1,9,"4,832.15"
9,10,osafrpsj,2,2,"9,812.81"


In [12]:
df2.limit(10).show()

+-----+---------+-------+-----+------------------+
|index|passwords|quarter|month|             price|
+-----+---------+-------+-----+------------------+
|    1| hchxqejg|      4|   12| 7243.510372272295|
|    2| gwvchnfp|      2|    6| 7404.993813933027|
|    3| ypyqefdp|      4|    7|1914.8197305207837|
|    4| pboiwwrf|      4|    1| 8157.257283019437|
|    5| xnuwoqxj|      3|    9|2394.5196122301913|
|    6| tcayjrpi|      2|    8|3887.9385116336994|
|    7| mjbktjdy|      1|    7| 7123.511051106608|
|    8| usgvieju|      2|   12|2721.7587590768753|
|    9| wkdhqjtx|      3|   11| 6250.906848644251|
|   10| kzittfob|      4|   12|2710.7156934238496|
+-----+---------+-------+-----+------------------+



## Join df1 and df2 on indices.

In [ ]:
df_index = df1.join(df2,df1.index == df2.index)
df_index = df1.join(df2,df1["index"] == df2["index"])

In [ ]:
%%time
df_index.cache()

In [ ]:
df_index.show(20)

In [ ]:
df_index.columns

## Join df1 and df2 on indices. (round 2)

In [ ]:
df_i2 = df1.alias("o")\
.join(df2.alias("t"),col("o.index") == col("t.index"))

# df_index = df1.join(df2,df1["index"] == df2["index"])

In [ ]:
%%time
df_i2.cache()

In [ ]:
df_i2.show(20)

In [ ]:
df_i2.columns

## Create a new column calculated from current column values.

In [14]:
df3 = df1\
.filter(col("month") == 9)\
.filter(col("month") == 8)

In [22]:
df4 = df1\
.filter((col("month") == 9) | 
        (col("month") == 8) |
        (col("quarter") == 3)
       )

In [15]:
df3.limit(8).toPandas()

,index,passwords,quarter,month,price


In [24]:
df4.limit(32).toPandas()

,index,passwords,quarter,month,price
0,1,clcnnvyv,2,8,"6,022.08"
1,4,dbpxfrqw,3,12,"8,172.45"
2,5,bdqlofhq,2,9,"3,720.03"
3,11,jkztjrrm,1,8,"6,926.03"
4,12,jdailwpz,3,9,"2,013.15"
5,15,lqgkktxc,3,1,"2,504.63"
6,18,toqcmpak,3,11,"8,211.26"
7,21,nzwehhzg,3,4,"2,402.05"
8,22,awqrcast,3,3,538.21
9,26,msfudwue,3,12,"2,483.86"


In [23]:
df4.count()

180

In [40]:
df5 = df4\
.groupBy("month", "quarter")\
.agg(avg("price"))\
.orderBy("month")

In [42]:
df5.count()

18

In [43]:
df6 = df5.filter(col("quarter") == 3)

In [46]:
df6.limit(10).toPandas()

,month,quarter,avg(price)
0,1,3,"6,097.54"
1,2,3,"7,020.26"
2,3,3,"5,117.60"
3,4,3,"4,486.07"
4,5,3,"3,399.59"
5,6,3,"5,939.97"
6,7,3,"6,539.49"
7,8,3,"5,642.59"
8,9,3,"3,891.82"
9,10,3,"4,920.09"


In [41]:
df5.limit(50).toPandas()

,month,quarter,avg(price)
0,1,3,"6,097.54"
1,2,3,"7,020.26"
2,3,3,"5,117.60"
3,4,3,"4,486.07"
4,5,3,"3,399.59"
5,6,3,"5,939.97"
6,7,3,"6,539.49"
7,8,3,"5,642.59"
8,8,2,"4,350.95"
9,8,4,"6,849.49"


In [37]:
# df4.groupby("quarter")\
# .agg(count("quarter")).show()

df4\
.withColumn("eps_avg",when((col("month") == 9),sum(df4.filter(col("month") == 9)['price'])).otherwise(0))

            
# .filter(col("quarter") == 3 & col("month") != 8
#     .agg(when(, count("quarter")))


AnalysisException: "grouping expressions sequence is empty, and '`index`' is not an aggregate function. Wrap '(CASE WHEN (`month` = CAST(9 AS BIGINT)) THEN sum(`price`) ELSE CAST(0 AS DOUBLE) END AS `eps_avg`)' in windowing function(s) or wrap '`index`' in first() (or first_value) if you don't care which value you get.;;\nAggregate [index#0L, passwords#1, quarter#2L, month#3L, price#4, CASE WHEN (month#3L = cast(9 as bigint)) THEN sum(price#4) ELSE cast(0 as double) END AS eps_avg#119]\n+- Filter (((month#3L = cast(9 as bigint)) || (month#3L = cast(8 as bigint))) || (quarter#2L = cast(3 as bigint)))\n   +- LogicalRDD [index#0L, passwords#1, quarter#2L, month#3L, price#4], false\n"

# df1 cont'd

In [48]:
df1.limit(10).toPandas()

,index,passwords,quarter,month,price
0,1,clcnnvyv,2,8,"6,022.08"
1,2,quioxrhy,1,11,161.25
2,3,nbysktmo,4,3,"7,117.66"
3,4,dbpxfrqw,3,12,"8,172.45"
4,5,bdqlofhq,2,9,"3,720.03"
5,6,dfyzprsc,1,1,"1,981.59"
6,7,tfuuzccz,1,4,"7,594.63"
7,8,apmdmwju,1,6,"6,312.08"
8,9,rftrcvdm,2,12,"2,623.78"
9,10,cdjeenfq,4,10,"3,660.70"


In [51]:
w_priceperquarter = (Window.orderBy(col("month")).rangeBetween(4, 6))


NameError: name 'Window' is not defined

In [50]:
df1.withColumn("avg", avg('price over quarter').show()

SyntaxError: unexpected EOF while parsing (<ipython-input-50-459a1909533c>, line 1)